<a href="https://colab.research.google.com/github/Vikadie/AI-repo/blob/master/NER_and_Entity_Linking_in_legal_documents_in_Bulgarian_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

In [ ]:
import os
from random import randint, seed
from time import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle

!pip install nose

from nose.tools import *

     |████████████████████████████████| 163kB 4.2MB/s 


In [ ]:
import tensorflow as tf

# NER and Entity Linking in unstructured legal documents in Bulgarian language

##### Final exam report

*Viktor Belchev - student*

*Deep Learning - Software University*

*February 2021*

### Abstract

Abstract

In all legal documents there is usage of citation of different laws or other juridical terms often hidden behind some abbreviations. While this aims to make the text shorter and clearer it is mostly causing troubles in understanding and translation to simple language not only to regular persons, but sometimes even lawyers and people with juridical background feel lost. Therefore, it often requires an additional research in the legal litterature. At this stage another problem might occur - the correct decoding of abbreviated term can become obstacle on top in correct decoding of the information.

In this paper, I will try to use the modern approach of Deep Learning to create a helpful tool that overcomes these problems. Using the state-of-the-art available models in the field of Natual Language Processing like BERT and an abbreviation list available at this stage, I will try to achieve an acceptable accuracy in this task for Bulgarian language that is known as combination of two different tasks: Named Entity Recognition and Entity Linking. 

### Introduction

Generally, in Natural Language Processing (further, NLP) the process of disambiguation of terms is known as Entity Linknig (further, EL), which goes hand in hand with another operation called Named Entity Recognition (further, NER). As explained by Iva Marinova in her **"Reconstructing NER Corpora: a Case Study on Bulgarian"** while in the field of Deep Learning 
these two related tasks are considered to be well covered in
NLP for Germanic, Romance and other language groups,
they are still under-resourced for the Slavic languages, especially from a multilingual perspective.

Usually, the order of application of both tasks is by starting with NER.

The purpose of NER is to tag words in a sentences based on some predefined tags, in order to extract some important information of the sentence, like for instanse names, geographical locations, dates, currency etc.
In NER, each token in the sentence will get tagged with a label, the label will tell the specific meaning of the token. In that way, through NER, we can analyze the sentence with more details and extract the important information.

There are two popular approaches for NER:
- multi-class classification based where NER is treated as a multi-class classification process, and we can use some text classification method to label the token.
- Conditional Random Field(CRF) based method labels the token taking context into account, then predicts sequences of labels for sequences of sentence token then get the most reasonable one. It is a probabilistic graphical model.

The identification of named entity mentions in texts is often implemented using a sequence tagger, where each token is labeled with an BIO tag, indicating whether the token begins a named entity — (B-), whether it is inside of a named entity (I-), or outside of a named entity (O-). This type of annotation has been proposed for the first time at CoNLL-2003 dataset created for NER (Tjong Kim Sang and De Meulder, 2003). There are other tag notation types. For instance, each token can be predicted with a tag indicated by B-(begin), I-(inside), E-(end), S-(singleton) of a named entity with its type, or O-(outside) of named entities. But, I will stick to BIO format of representation for simplicity.

Entity linking can be applied rigth after the NER task is performed althought in some papers on this topic there is proposal to do it in parallel (jointly) for each token, so that each subtask benefits from the partial output of the other subtask, and thus alleviate error propagations that are unavoidable in pipeline settings. 
Generally, EL is the task of mapping words from text (e.g. names of persons, locations and organizations) to entities from the target knowledge base. For this pupose I use a document containing most of the existing abbreviations used in legal documents.

### NER

Usually, no matter the specific task, Deep Learning models creation is based on big data for training, validation and test. For Bulgarian language generally such data could be available if we start scraping web pages, which is huge amount of work. The problem afterwards is hidden in the effectiveness of the model created itself. Luckily, after the publication of the famous paper called "Attention is all you need" by Vaswani and the "appearance" of *Transformer*, there is a huge advancement compared to previous usage of recurrence (RNN), Bidirectional Lont-Short Term Memory units (BiLSTM), convolutions (CNN) and CRF. Transformer utilizes stacked self-attention and pointwise, fully connected layers to build basic blocks for encoder and decoder.  Experiments on various tasks show Transformers to be superior in quality while requiring significantly less time to train.

Based mostly on transformer, it already exists pre-trained models that provide results pretty close to humans on some general tasks. Some of the most used methods are ELMo(Embeddings from Language Models), OpenAI GPT (Generative Pre-trained Transformer), BERT (Bidirectional Encoder Representations from Transformers)... 

It is important to underline that these state-of-the-art models use *hybrid representation* of text (embeddings) in low dimensional real-valued dense vectors. It is called *hybrid* as it uses *Word-level* and *Character-level* representation along with some additional features, where each dimensions represents a latent feature. This way it also captures the semantic and syntactic properties of words, but also the context for each word.

In recent years, the advancements of NLP in general and NER in particular has been greatly influenced by deep transfer learning methods capable of creating contextual representations of words, to the extent that many of the state-of-the-art NER systems mainly differ from one another on the basis of how these contextual representations are created. Using such models, sequence tagging tasks are often approached one sentence at a time, essentially discarding any information available in the broader surrounding context, and there is only little recent study on the use of cross-sentence context – sentences around the sentence of interest – to improve sequence tagging performance.

Precisely for the fact of using this cross-sentence context, but also with the advantage to be pre-trained on Bulgarian texts, in this notebook, I focus on the recent BERT deep transfer learning models based on self-attention and the transformer architecture. BERT uses a fixed-size window that limits the amount of text that can be input to the model at one time. The model maximum window size, or maximum sequence length, is fixed during pre-training, with 512 wordpieces a common choice. This window fits dozens of typical sentences of input at a time, allowing the inclusion of extensive sentence context.

There are many advantages that pushed me towards usage of BERT. To enumerate some, I would say that it provides:
1. quicker development
2. Overcome the problem of missing data for training, which is generally the case for Bulgarian
3. state-of-the-art better results

On the other hand, there are some disadvantages, like:
1. it is very large. The LARGE version of BERT would provide better results, but unfortunately that would require bigger computational ability and time.
2. Even when using the BASE version, it remains slow for fine-tuning.
3. The multilingual version that I need to use cannot be disitilled - the vocabulary used for fine-tuning of BERT must remain the original one.
4. It uses a specific and a bit complicated jargon (domain-specific language), meaning that the tokenization with BERT should be done with BERT Tokenizer.

The last two disadvantages represent in fact the specifity and maybe the strength of BERT. Its vocabulary is indeed fixed, but it has the capability to break down the unknown word into subwords and makes a token out of each subword (if subword exists in the vocabulary). In case the subword do not exist in the vocabulary it can continue spliting it into subwords down to a character level. To recognize the subword it prepends it with "##" flag, exceot for the first subword.

That would creat a problem with labeling. Generally, in the test and train part each word is tagged. If an unknown word is splitted to subword, a specific tag should be used for it, that would indicate that the tag valid for this word would be the one given to the first subword (original word).

### Entity Linking

The Entity Linking (EL) process transforms amiguous textual mention to a unique identifier by looking at the context in which the mention occurs. Thus it can be looked as 2 step process after the NER:
1. Creation of Entity Linker - list of candidates for each mention generation
2. Reduce the list to the final ID that represents the correct name.

This is generally the method used in `spacy` module.

Another option used for this is used in `deeppavlop` module where:
1. NER is fed to tf-idf Vectorizer and the resulting sparse vector is converted to dense vector.
2. A library called Flair is used to find the k-nearest neighbours for tf-idf vector in the matrix where each row is a tf-idf vectors of words in entity titles.
3. entities are ranked by number of relations in wikidata.

### Conclusion and Future Work

Conclusion

### Ressources

**Reconstructing NER Corpora: a Case Study on Bulgarian** - Iva Marinova, Laska Laskova, Petya Osenova, Kiril Simov, Alexander Popov - Proceedings of the 12th Conference on Language Resources and Evaluation (LREC 2020), pages 4647–4652, Marseille, 11–16 May 2020

**Tuning Multilingual Transformers for Named Entity Recognition on
Slavic Languages** - Mikhail Arkhipov, Maria Trofimova, Yuri Kuratov, Alexey Sorokin - Neural Networks and Deep Learning Laboratory, Moscow Institute of Physics and Technology, Faculty of Mathematics and Mechanics, Moscow State University - Proceedings of the 7th Workshop on Balto-Slavic Natural Language Processing, pages 89–93, Florence, Italy, 2 August 2019. - https://www.aclweb.org/anthology/W19-3712.pdf


**BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding** - Jacob Devlin, Ming-Wei Chang, Kenton Lee and Kristina Toutanova - Google AI Language, 24 May 2019 - https://arxiv.org/pdf/1810.04805.pdf


**Attention Is All You Need** - Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin, 6 Dec 2017 - https://arxiv.org/pdf/1706.03762.pdf

**A Survey on Deep Learning for Named Entity Recognition** - Jing Li, Aixin Sun, Jianglei Han, and Chenliang Li - IEEE TRANSACTIONS ON KNOWLEDGE AND DATA ENGINEERING, 18 Mar 2020 - https://arxiv.org/pdf/1812.09449v3.pdf

**Zero-Resource Cross-Domain Named Entity Recognition** - Zihan Liu, Genta Indra Winata, Pascale Fung - Center for Artificial Intelligence Research (CAiRE), Department of Electronic and Computer Engineering, The Hong Kong University of Science and Technology, Clear Water Bay, Hong Kong, 19 May 2020 - https://arxiv.org/pdf/2002.05923.pdf

**Exploring Cross-sentence Contexts for Named Entity Recognition with BERT** - Jouni Luoma, Sampo Pyysalo - Turku NLP group, University of Turku, Finland, 2 Jun 2020 - https://arxiv.org/pdf/2006.01563v1.pdf

**NER with BERT in Action** - Bill Huang - July 30, 2019- https://medium.com/@yingbiao/ner-with-bert-in-action-936ff275bc73

**Deep contextualized word representations** - Matthew E. Peters, Mark Neumann, Mohit Iyyer, Matt Gardner, Christopher Clark, Kenton Lee, Luke Zettlemoyer - Allen Institute for Artificial Intelligence and Paul G. Allen School of Computer Science & Engineering, University of Washington, 22 Mar 2018 - https://arxiv.org/pdf/1802.05365.pdf


**Improving Language Understanding by Generative Pre-Training** - Alec Radford, Karthik Narasimhan, Tim Salimans, Ilya Sutskever - Open AI - https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf